In [38]:
EG = "/Users/joregan/Playing/merged_annotations/hsi_4_0717_209_002.json"

In [39]:
import json

In [40]:
with open(EG) as inf:
    data = json.load(inf)

In [9]:
SAMPLE = """
{
  "2": {
    "general": {
      "id": 2,
      "start": 5.970287890159987,
      "end": 9.551266621561616
    },
    "high_level": {
      "topic_name": "reference_real",
      "current_topic": "vase",
      "topic_change": false,
      "topic_duration_id": 0,
      "spatial_reference": "fourth vase",
      "referenced_object": [
        "vase"
      ]
    },
    "low_level": {
      "resolved_references": {
        "the fourth vase": "vase",
        "it": "vase"
      },
      "spatial_relationships": [],
      "resolved_adverbs": []
    },
    "snippet": "But I'm wondering about the fourth vase. Where is it? Did you break it?"
  }
}
"""

```json
      "resolved_references_indices": {
        "it": [2, 4]
      }
```

In [ ]:
def load_tsv(filename):
    data = []
    with open(filename) as inf:
        for line in inf.readlines():
            parts = line.strip().split("\t")
            data.append({
                "start": float(parts[0]),
                "end": float(parts[1]),
                "word": parts[2]
            })
    return data

In [47]:
def slice_tsv_data(data, start, end):
    ret = []
    for datum in data:
        if type(datum["start"]) is str:
            datum["start"] = float(datum["start"])
        if type(datum["end"]) is str:
            datum["end"] = float(datum["end"])
        if datum["start"] >= start and datum["end"] <= end:
            ret.append(datum)
        elif datum["end"] > end:
            return ret
    return ret

In [50]:
def clean_text(text):
    return " ".join([x.lower().strip(".,;?!") for x in text.split(" ")])

In [51]:
from pathlib import Path

TSVS = Path("/Users/joregan/Playing/hsi/word_annotations/")
JSON = Path("/Users/joregan/Playing/merged_annotations/")

In [93]:
def get_indices(needle, haystack, checkpos=True):
    ret = []
    nwords = [x.lower().strip(",?.;:") for x in needle.split(" ")]
    hwords = [x.lower().strip(",?.;:") for x in haystack.split(" ")]
    nwordspos = nwords[:-1] + [f"{nwords[-1]}'s"]
    nlen = len(nwords)

    for i in range(len(hwords)):
        if hwords[i:i+nlen] == nwords:
            ret.append((i, i+nlen))
        elif checkpos and hwords[i:i+nlen] == nwordspos:
            ret.append((i, i+nlen))
    return ret

In [ ]:
def get_tsv_for_segment(segment, tsv_data):
    assert "general" in segment, "Missing key 'general'"
    assert "start" in segment["general"], "Missing key 'start'"
    assert "end" in segment["general"], "Missing key 'end'"

    start = segment["general"]["start"]
    end = segment["general"]["end"]

    tsv = slice_tsv_data(tsv_data, start, end)
    tsv_words = " ".join([x["word"] for x in tsv])

    if segment["snippet"] != tsv_words:
        cleaned_snippet = clean_text(segment["snippet"])
        cleaned_text = clean_text(tsv_words)

        if cleaned_snippet not in cleaned_text:
            print("🙀 mismatch:", "🖇️", segment["snippet"], "🎧", tsv_words, cleaned_text.find(cleaned_snippet))
            return []
        else:
            idxes = get_indices(cleaned_snippet, cleaned_text)
            assert len(idxes) == 1
            tsv = tsv[idxes[0][0]:idxes[0][1]]
            tsv_words = " ".join([x["word"] for x in tsv])
            cleaned_text = clean_text(tsv_words)
            assert cleaned_snippet == cleaned_text, f"🖇️ {cleaned_snippet} 🎧 {cleaned_text}"
    return tsv
        

In [109]:
for seg in data:
    print(get_tsv_for_segment(data[seg], ttsv))

[{'start': 2.835, 'end': 3.055, 'word': 'Yeah.'}]
[{'start': 4.395, 'end': 4.655, 'word': 'Good.'}]
[{'start': 5.99, 'end': 6.31, 'word': 'But'}, {'start': 6.31, 'end': 6.51, 'word': "I'm"}, {'start': 6.51, 'end': 6.89, 'word': 'wondering'}, {'start': 6.89, 'end': 7.09, 'word': 'about'}, {'start': 7.09, 'end': 7.21, 'word': 'the'}, {'start': 7.21, 'end': 7.53, 'word': 'fourth'}, {'start': 7.53, 'end': 8.05, 'word': 'vase.'}, {'start': 8.05, 'end': 8.33, 'word': 'Where'}, {'start': 8.33, 'end': 8.43, 'word': 'is'}, {'start': 8.43, 'end': 8.69, 'word': 'it?'}, {'start': 8.69, 'end': 8.95, 'word': 'Did'}, {'start': 8.95, 'end': 9.03, 'word': 'you'}, {'start': 9.03, 'end': 9.29, 'word': 'break'}, {'start': 9.29, 'end': 9.43, 'word': 'it?'}]
[{'start': 13.816, 'end': 13.836, 'word': 'I'}, {'start': 13.836, 'end': 14.036, 'word': 'have'}, {'start': 14.036, 'end': 14.416, 'word': 'three'}, {'start': 14.416, 'end': 14.896, 'word': 'now.'}, {'start': 14.896, 'end': 15.076, 'word': 'I'}, {'start

In [92]:
get_indices("the needle", "the needle is the needle's problem", False)

[(0, 2)]